<a href="https://colab.research.google.com/github/alexmjn/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np

In [5]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-12-09 20:37:38--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-12-09 20:37:39 (286 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [3]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?
republican,n,y,n,y,y,n,n,n,n,n,?,?,y,y,n,n
republican,n,y,n,y,y,y,n,n,n,n,y,?,y,y,?,?
democrat,n,y,y,n,n,n,y,y,y,n,n,n,y,n,?,?
democrat,y,y,y,n,n,y,y,y,?,y,y,?,n,n,y,?
republican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,?,n,?
democrat,y,n,y,n,n,y,n,y,?,y,y,y,?,n,n,y
democrat,y,?,y,n,n,n,y,y,y,n,n,n,y,n,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,?,y,y,n,n
democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
democrat,y,y,y,n,n,?,y,y,n,n,y,n,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,?,?,y,y
democrat,y,?,y,n,n,n,y,y,y,n,n,?,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,

In [8]:
voting_data = pd.read_csv('house-votes-84.data', 
                          header = None, 
                          names = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'], na_values = "?")
print(voting_data.shape)
voting_data.head()

(435, 17)


party handicapped-infants water-project  ... crime duty-free south-africa
0  republican                   n             y  ...     y         n            y
1  republican                   n             y  ...     y         n          NaN
2    democrat                 NaN             y  ...     y         n            n
3    democrat                   n             y  ...     n         n            y
4    democrat                   y             y  ...     y         y            y

[5 rows x 17 columns]

In [0]:
voting_data = voting_data.replace({'n': 0, 'y':1})
dem = voting_data[voting_data['party'] == 'democrat']
rep = voting_data[voting_data['party'] == 'republican']

In [0]:
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

In [11]:
dem.head()

party  handicapped-infants  water-project  ...  crime  duty-free  south-africa
2  democrat                  NaN            1.0  ...    1.0        0.0           0.0
3  democrat                  0.0            1.0  ...    0.0        0.0           1.0
4  democrat                  1.0            1.0  ...    1.0        1.0           1.0
5  democrat                  0.0            1.0  ...    1.0        1.0           1.0
6  democrat                  0.0            1.0  ...    1.0        1.0           1.0

[5 rows x 17 columns]

In [12]:
rep.head()

party  handicapped-infants  ...  duty-free  south-africa
0   republican                  0.0  ...        0.0           1.0
1   republican                  0.0  ...        0.0           NaN
7   republican                  0.0  ...        NaN           1.0
8   republican                  0.0  ...        0.0           1.0
10  republican                  0.0  ...        0.0           0.0

[5 rows x 17 columns]

In [13]:
dem.describe()

handicapped-infants  water-project  ...   duty-free  south-africa
count           258.000000     239.000000  ...  251.000000    185.000000
mean              0.604651       0.502092  ...    0.637450      0.935135
std               0.489876       0.501045  ...    0.481697      0.246956
min               0.000000       0.000000  ...    0.000000      0.000000
25%               0.000000       0.000000  ...    0.000000      1.000000
50%               1.000000       1.000000  ...    1.000000      1.000000
75%               1.000000       1.000000  ...    1.000000      1.000000
max               1.000000       1.000000  ...    1.000000      1.000000

[8 rows x 16 columns]

In [14]:
rep.describe()

handicapped-infants  water-project  ...   duty-free  south-africa
count           165.000000     148.000000  ...  156.000000    146.000000
mean              0.187879       0.506757  ...    0.089744      0.657534
std               0.391804       0.501652  ...    0.286735      0.476168
min               0.000000       0.000000  ...    0.000000      0.000000
25%               0.000000       0.000000  ...    0.000000      0.000000
50%               0.000000       1.000000  ...    0.000000      1.000000
75%               0.000000       1.000000  ...    0.000000      1.000000
max               1.000000       1.000000  ...    1.000000      1.000000

[8 rows x 16 columns]

In [15]:
ttest_ind(dem["religious-groups"], rep["religious-groups"], nan_policy = "omit")

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

The mean for Democrats was 47.7% support. The mean for Republicans was 89.8% support. The t-statistic of -9.73 and p-value of 2.39 * 10^-20 suggests that this difference would essentially never occur purely due to chance; as it's less than .05, We can reject the null hypothesis that the true difference between Democrat and Republican support is 0 and conclude that Republicans have greater support for this religious groups bill.

In [0]:
def ttest_congress(bill):
  return ttest_ind(dem[bill], rep[bill], nan_policy = 'omit')

In [17]:
ttest_congress("religious-groups")

Ttest_indResult(statistic=-9.737575825219457, pvalue=2.3936722520597287e-20)

In [18]:
ttest_congress("synfuels")

Ttest_indResult(statistic=8.293603989407588, pvalue=1.5759322301054064e-15)

The t-statistic is 8.29 and the p-value is 1.58 * 10^-15 < .05. Thus we can reject the null hypothesis that the true difference is 0 and conclude that Democrats have greater support for this bill than Republicans. 

In [20]:
ttest_congress("water-project")

Ttest_indResult(statistic=-0.08896538137868286, pvalue=0.9291556823993485)

Here the t-statistic is just -.09 and the p-value is .929. Thus there is no evidence that allows us to reject the null hypothesis. 